# 01 — Preprocess & Visualize Bamberg Noise Data

This notebook will:

1. Load Bamberg’s boundary and noise point data  
2. Clean the points (db-range, accuracy)  
3. Generate a 100 m square grid, clipped to the city  
4. Count how many points fall in each cell  
5. Plot a static choropleth with GeoPandas  
6. Create an interactive Folium map  


In [3]:
import numpy as np
import geopandas as gpd
import folium
from shapely.geometry import box

# File paths
POINTS_FP = "../data/raw/Germany_Bayern_Bamberg.points.geojson"
BOUNDARY_FP = "../data/raw/Germany_Bayern_Bamberg.areas.geojson"

# CRS codes
WGS84_CRS = 4326
METRIC_CRS = 25832  # UTM zone 32N

# Grid
CELL_SIZE = 100  # meters

In [4]:
# Cell 2 –– load & filter noise points


def clean_points(gdf, db_col="noise_level", min_db=30, max_db=120, max_acc=15):
    """Keep only points with realistic noise levels & accuracy."""
    mask = (gdf[db_col] >= min_db) & (gdf[db_col] <= max_db)
    gdf = gdf[mask]
    if "accuracy" in gdf.columns:
        gdf = gdf[gdf["accuracy"] <= max_acc]
    return gdf


# load
boundary = gpd.read_file(BOUNDARY_FP)
points = gpd.read_file(POINTS_FP)

print(f"Raw points: {len(points):,}")
print("Columns available:", points.columns.tolist())

# clean using the correct column name
points = clean_points(points, db_col="noise_level")
print(f"Filtered points: {len(points):,}")

Raw points: 1,336
Columns available: ['pk_track', 'time_ISO8601', 'time_epoch', 'time_gps_ISO8601', 'time_gps_epoch', 'noise_level', 'speed', 'orientation', 'accuracy', 'geometry']
Filtered points: 950


In [5]:
# Cell 3: project points into metres so our 100 m grid is accurate

# we choose EPSG:25832 (UTM zone 32N), which uses metres
points_m = points.to_crs(epsg=25832)

print("✓ Points now in EPSG:25832 (units = metres)")
print("Bounds (metres):", points_m.total_bounds)

✓ Points now in EPSG:25832 (units = metres)
Bounds (metres): [ 631011.26767521 5526504.36829996  643419.87081591 5542474.52891367]


In [6]:
# Cell 4: build a fishnet of 100 m cells covering all measurements

# 1) Compute min/max X/Y in metres
minx, miny, maxx, maxy = points_m.total_bounds

# 2) Create all square cells
step = 100  # 100 m
xs = np.arange(minx, maxx + step, step)
ys = np.arange(miny, maxy + step, step)

cells = []
for x in xs[:-1]:
    for y in ys[:-1]:
        cells.append(box(x, y, x + step, y + step))

grid_m = gpd.GeoDataFrame(geometry=cells, crs=points_m.crs)
print(f"✓ Built {len(grid_m)} raw grid cells (100 m each).")

✓ Built 20000 raw grid cells (100 m each).


In [7]:
# Cell 5: spatial‐join & count

# 1) assign each point to its grid cell
joined = gpd.sjoin(points_m, grid_m, how="inner", predicate="within")

# 2) count points per cell index
counts = joined.groupby("index_right").size().rename("pt_count")

# 3) attach back to our grid
grid_m["pt_count"] = counts
grid_m["pt_count"] = grid_m["pt_count"].fillna(0).astype(int)

print("✓ Counted points in each cell.")
print(grid_m["pt_count"].describe())

✓ Counted points in each cell.
count    20000.000000
mean         0.047400
std          2.730353
min          0.000000
25%          0.000000
50%          0.000000
75%          0.000000
max        250.000000
Name: pt_count, dtype: float64


In [ ]:
# Cell 6: interactive map with folium
from folium.plugins import MarkerCluster
import branca.colormap as cm

# 1) reproject back to WGS84 for Folium
grid_wgs = grid_m.to_crs(epsg=4326)
pts_wgs = points.to_crs(epsg=4326)

# 2) build a simple linear colormap from 30 dB→120 dB
colormap = cm.LinearColormap(
    ["green", "yellow", "red"], vmin=30, vmax=120, caption="Noise level (dB)"
)

# 3) pick the map centre
centre = pts_wgs.unary_union.centroid

# 4) create the map
m = folium.Map(
    location=[centre.y, centre.x], zoom_start=13, tiles="OpenStreetMap"
)

# 5) add the colormap legend
colormap.add_to(m)

# 6) draw your 100 m grid (no fill, thin grey lines)
folium.GeoJson(
    grid_wgs,
    style_function=lambda feat: {
        "fillOpacity": 0,
        "weight": 0.3,
        "color": "#444444",
    },
    name="100 m grid",
).add_to(m)

# 7) cluster & plot each noise point
marker_cluster = MarkerCluster(name="Noise readings").add_to(m)

for _, row in pts_wgs.iterrows():
    db = row["noise_level"]
    time = row["time_ISO8601"]
    lat, lon = row.geometry.y, row.geometry.x

    # radius scaled 1px per 10 dB, clipped between 3 and 10px
    radius = min(max((db - 30) / 10, 3), 10)

    folium.CircleMarker(
        location=[lat, lon],
        radius=radius,
        color=colormap(db),
        fill=True,
        fill_color=colormap(db),
        fill_opacity=0.8,
        popup=folium.Popup(
            f"<b>Noise:</b> {db:.1f} dB<br><b>Time:</b> {time}", max_width=200
        ),
    ).add_to(marker_cluster)

# 8) layer control
folium.LayerControl(collapsed=False).add_to(m)

m  # renders live in Jupyter

NameError: name 'S' is not defined